# Count-Min Sketch
---

The Count-Min (CM) sketch is a probabilistic data structure that provides
a lossy form of compression for large count/frequency datasets.
It is typically used for streaming data. At the heart of the CM sketch
is hashing. The CM sketch uses a set of hash functions with corresponding,
constant size, hash tables. These hash functions are independent from one
another. Since the hash functions are independent, each distributes
data differently within its hash table. This independent hashing redundancy allows
CM sketches to achieve a high degree of lossy compression while still 
producing quality estimates of the original data.

### Internals
---
The core data storage structure within a CM sketch is a $w$ * $d$ table, $\text{count}$. $w$ is given by $w = \left\lceil\frac{e}{\epsilon}\right\rceil$ and d is given by $d = \ln\left(\frac{1}{\delta}\right)$. $\epsilon$ is the additive error factor that a result will be within with probability $1-\delta$.

<img src="./img/cm_internal_table.png" width="400" />

Each row in the table is used as the hash table for one of the $1..d$ hash functions. When we add an event to the sketch, its count is added to each row.

<img src="./img/cm_adding_event.png" width="400" />

### Operations
---
#### Point Query $Q(i)$
A point query is the estimation of $a_i$ from the original data.

<img src="./img/cm_point_q.png" width="400" />
$$Q(i) = \min_j\text{count}[j, h_j(i)]$$

#### Range Query $Q(l, r)$
A range query from $l..r$ is the estimation of the sum over that range.
$$Q(l,r) = \sum_{i=l}^r a_i$$
To accuratly calculate a range query, $log(n)$ sketches must be kept; one for each set of dyadic ranges spanning $1..n$.

#### Inner Product $Q(\boldsymbol{a}, \boldsymbol{b})$
The inner product between two arrays can be estimated using a sketch for each array and taking the minimum row-wise inner product.
$$Q(\boldsymbol{a}, \boldsymbol{b}) = \min_j\sum_{k=1}^n\text{count}_a[j, k]*\text{count}_b[j, k]$$

In [25]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns # visualizations
import os
import glob
import time
import sys
import mmh3

In [26]:
# naively count frequencies

class dictionary():
    
    def __init__(self):
        self.dictionary = {}
        self.nbytes = sys.getsizeof(self)
    
    def getsize(self):
        print("Dictionary is Size: {} Bytes\n".format(self.nbytes))
        
    def add(self,token):
        if token in self.dictionary:
            self.dictionary[token] += 1
        else:
            self.dictionary[token] = 1
        self.nbytes = sys.getsizeof(self.dictionary)
        
    def timed_update(self,tokenlist):
        startsize = self.nbytes
        start = time.time()
        for token in tokenlist:
            self.add(token)
        end = time.time() - start
        dsize = self.nbytes - startsize
        print("Time Elapsed: {} Seconds \n".format(end))
        print("Change In Memory: {} Bytes\n".format(dsize))
    
    def estimate(self,token):
        try:
            return self.dictionary[token]
        except:
            print("Error: Token Not Found \n")

In [27]:

class CountMinSketch():
    
    def __init__(self,indexes=2**5,hashfuncs=2**5):
        self.N = indexes
        self.M = hashfuncs

        self.seeds = np.arange(hashfuncs).tolist()
        self.table = np.zeros((self.M,self.N))
        self.hashes = [self._genhash(seed) for seed in self.seeds]
        self.nbytes = sys.getsizeof(self.table) + sys.getsizeof(self.hashes)
        
    def _genhash(self,prime):
        def hash_fn(val):
            index = mmh3.hash(val,seed=prime)
            return index%self.M
        return hash_fn

    def getsize(self):
        print("Sketch is Size: {} Bytes\n".format(self.nbytes))
        
    def add(self, val):      
        for ix in range(0, self.N):
            h = self.hashes[ix](val)
            self.table[ix][h] += 1

            
    def timed_update(self,valuelist):
        start = time.time()
        for value in valuelist:
            self.add(value)
        end = time.time() - start
        dsize = sys.getsizeof(self.table) + sys.getsizeof(self.hashes)
        print("Time Elapsed: {} Seconds \n".format(end))
        print("Memory Useage: {} Bytes\n".format(dsize))
                              
                              
       
    def count(self,val):
        # Helper Function
        vals = []
        for ix in range(0, N):
            h = self.hashes[ix](val)
            vals.append(self.table[h][ix])
        return vals
            
    def estimate(self, value):
   
        results = []
        for ix in range(0, N):
            h = self.hashes[ix](value)
            c = self.table[h][ix]
            results.append(c)
        return min(results)

In [28]:
class changeDetection():
    def __init__(self,sketchset,w):
        self.sketchset = sketchset
        self.w = w
        self.MA = None
        
    def update(self,sketch):
        self.sketchset.append(sketch)
        if len(self.sketchet >self.w):
            self.sketchset.pop(0)
        self.calcmovingAv()
            
    def calcmovingAv(self):
        tableav = 0
        for i,sketch in enumerate(self.sketchset):
            tableav += sketch.table
        tableav /= len(self.sketchset)
        self.MA = tableav
        
    def indmovingAv(self,val):
        # Helper Function
        mavals = []
        for ix in range(0, N):
            h = self.sketchset[-1].hashes[ix](val)
            mavals.append(self.MA[h][ix])
        return mavals
    
    def bucketDist(self,value,idx):
        # Helper Function
        sketch = self.sketchset[idx]
        K = np.sum(sketch.table > 0)
        counts = sketch.count(vals)
        vals = (counts - self.indmovingAv(value))
        vals = [value/(1 - 1/K) for value in vals]
        v_a  = np.median(vals)
        return counts,v_a
    
    def isAttack(self,value,beta):
        # given a set of values, compute their respective counts and determine the
        # "Change" in the count frequency relative to their respective bins to dermine 
        # an anomoly
        
        # If the ID post count exceeds the criteria from the previous sketch
        # then it qualifies as an attack
        counts,variance = self.bucketDist(value,-1)
        mean = self.MA
        crit = mean + beta*variance
        for count in counts:
            if count > crit:
                print("Found Anomoly: {},{}".format(count,crit))
                return True
          

In [45]:
class BinaryClassification():
    def __init__(self,A,B,update=False):
        # Arguments are either:
            #update = True : two respective lists of words from different datasets
            
            #update = False: Two previously instatiated instances of CountMinSketch()
        self.streamA = CountMinSketch()
        self.streamB = CountMinSketch()
        if update:
            self.streamB.timed_update(B)
            self.streamA.timed_update(A)
        else:
            self.streamA = A
            self.streamB = B
      
        
        
    def _dotProduct(self,tableA,tableB):
        return np.min(tableA@tableB.T)
        
        
    def classify(self,subsketch):
        x = self._dotProduct(subsketch.table,self.streamA.table)
        y = self._dotProduct(subsketch.table,self.streamB.table)
        print(x,y)
        print(x-y)
        if x > y:
            print("Words Originated from A")
        else:
            print("Words Originated from B")

In [30]:
fruitfly = np.load("fruitfly.npy")
human = np.load("human1.npy")

In [31]:
genomedict = dictionary()
genomedict.timed_update(human)

Time Elapsed: 1.96034836769104 Seconds 

Change In Memory: 83886120 Bytes



In [32]:
countsize = len(fruitfly)

In [33]:
len(fruitfly)
human = human[:len(fruitfly)]
len(human)

156246

In [34]:
genomesketch = CountMinSketch()
genomesketch.timed_update(human)

Time Elapsed: 4.866109132766724 Seconds 

Memory Useage: 8648 Bytes



In [35]:
fruitflysketch = CountMinSketch()
fruitflysketch.timed_update(fruitfly)

Time Elapsed: 4.850249767303467 Seconds 

Memory Useage: 8648 Bytes



In [36]:
 np.sum(genomesketch.table) - np.sum(fruitflysketch.table)

0.0

In [41]:
subhuman = human[::2]
subfly = fruitfly[::2]
SH_sketch = CountMinSketch()
SH_sketch.timed_update(subhuman)
FF_sketch = CountMinSketch()
FF_sketch.timed_update(subfly)

Time Elapsed: 2.4383835792541504 Seconds 

Memory Useage: 8648 Bytes

Time Elapsed: 2.404276132583618 Seconds 

Memory Useage: 8648 Bytes



In [46]:
fly_or_human = BinaryClassification(genomesketch,fruitflysketch)

In [47]:
fly_or_human.classify(SH_sketch)

381353392.0 381370583.0
-17191.0
Words Originated from B


In [48]:
fly_or_human.classify(FF_sketch)

381374195.0 381374906.0
-711.0
Words Originated from B
